# Piping a prompt, model, and output parser

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [3]:
chat = ChatOpenAI(
    model = 'gpt-4',
    temperature=0,
    seed=25,
    max_completion_tokens=100
)

In [4]:
instructions = CommaSeparatedListOutputParser().get_format_instructions()
instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [20]:
chat_prompt = ChatPromptTemplate.from_messages([('human', 'I want the list of 10 tourist places in {place}.\n' + instructions)])

In [21]:
print(chat_prompt.messages[0].prompt.template)

I want the list of 10 tourist places in {place}.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [22]:
output_parser = CommaSeparatedListOutputParser()

In [23]:
chain = chat_prompt | chat | output_parser

In [24]:
chain.invoke({'place': 'Delhi'})

['Red Fort',
 'Qutub Minar',
 'India Gate',
 'Lotus Temple',
 "Humayun's Tomb",
 'Jama Masjid',
 'Akshardham Temple',
 'Rashtrapati Bhavan',
 'Jantar Mantar',
 'Raj Ghat']